In [3]:
#Common libraries
import sys
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Libraries for Machine Learning
from sklearn import model_selection
from sklearn.metrics import classification_report, accuracy_score
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam